## Start

In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)



___
### Tags: #Source #Support

### Links: 
___

In [6]:
query_text = """--sql
CREATE TABLE db1.requests_st_support
(
    `report_date` Date,
    `ticket_id` Int32,
    `intercom_id` String,
    `camera_id` String,
    `server_stream_url` String,
    `category` String,
    `version_os` String,
    `version_app` String 
)
ENGINE = S3('https://storage.yandexcloud.net/dwh-asgard/requests_st_support/year=*/month=*/*.csv','CSVWithNames')
SETTINGS format_csv_delimiter = ';'
"""

ch.query_run(query_text)

DatabaseError: HTTPDriver for https://rc1a-eflrt968scbm232q.mdb.yandexcloud.net:8443 received ClickHouse error code 57
 Code: 57. DB::Exception: Table db1.requests_st_support already exists. (TABLE_ALREADY_EXISTS) (version 25.3.3.42 (official build))


In [8]:
query_text = """--sql
CREATE TABLE db1.requests_st_support_ch
    (
        `report_date` Date,
        `ticket_id` Int32,
        `intercom_id` String,
        `camera_id` String,
        `server_stream_url` String,
        `category` String,
        `version_os` String,
        `version_app` String,
        `device_type` String
    )
    ENGINE = MergeTree()
    ORDER BY report_date
    SETTINGS format_csv_delimiter = ';'
"""

ch.query_run(query_text)

DatabaseError: HTTPDriver for https://rc1a-eflrt968scbm232q.mdb.yandexcloud.net:8443 received ClickHouse error code 57
 Code: 57. DB::Exception: Table db1.requests_st_support_ch already exists. (TABLE_ALREADY_EXISTS) (version 25.3.3.42 (official build))


In [9]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.requests_st_support_mv
    REFRESH EVERY 1 DAY OFFSET 3 HOUR TO db1.requests_st_support_ch AS
    SELECT 
        `report_date`, 
        `ticket_id`, 
        `intercom_id`, 
        `camera_id`, 
        `server_stream_url`, 
        `category`, 
        `version_os`, 
        `version_app`,
        if(`intercom_id` != '', 'intercom','camera') as device_type
    FROM `db1`.`requests_st_support`
    """

ch.query_run(query_text)

DatabaseError: HTTPDriver for https://rc1a-eflrt968scbm232q.mdb.yandexcloud.net:8443 received ClickHouse error code 57
 Code: 57. DB::Exception: Table db1.requests_st_support_mv already exists. (TABLE_ALREADY_EXISTS) (version 25.3.3.42 (official build))


___
## Tools
___
### query


In [2]:
query_text = """--sql
    SELECT
        *
    FROM db1.requests_st_support_ch
    ORDER BY report_date desc
    limit 10
    """

ch.query_run(query_text)


report_date,ticket_id,intercom_id,camera_id,server_stream_url,category,version_os,version_app,device_type
date,i64,str,str,str,str,str,str,str
2025-07-23,197778,"""2148132017""","""""","""msk-v06.sputnik.systems""","""MOB|Video: видео нет/не работа…","""iOS""","""4.34.0(12)""","""intercom"""
2025-07-23,197781,"""2235144474""","""""","""msk-v10.sputnik.systems""","""""","""iOS""","""4.34.0(12)""","""intercom"""
2025-07-23,197780,"""2422252426""","""""","""msk-v01.sputnik.systems""","""""","""Android""","""6.21.1(417)""","""intercom"""
2025-07-23,197779,"""2101116473""","""""","""msk-v02.sputnik.systems""","""""","""iOS""","""4.33.0(3)""","""intercom"""
2025-07-23,197774,"""2018109524""","""""","""msk-v03.sputnik.systems""","""MOB|Storage: видеоархива нет/н…","""Android""","""6.21.1(417)""","""intercom"""
2025-07-23,197777,"""2435157434""","""""","""v07.sputnik.systems""","""MOB|Video: видео не работает/н…","""Android""","""6.21.1(417)""","""intercom"""
2025-07-23,197776,"""2012105335""","""""","""v07.sputnik.systems""","""MOB: приложение не работает, M…","""Android""","""6.21.1(417)""","""intercom"""
2025-07-23,197775,"""2012106797""","""""","""msk-v03.sputnik.systems""","""MOB: приложение не работает, M…","""Android""","""6.18.2(409)""","""intercom"""
2025-07-23,197768,"""2447161753""","""""","""msk-v03.sputnik.systems""","""Другое.""","""Android""","""6.20.0(413)""","""intercom"""


In [ ]:
category

### delete a part


In [ ]:
query_text = """--sql
    ALTER TABLE db1.requests_st_support_ch DELETE WHERE report_date = '2025-07-17'
    """

ch.query_run(query_text)


shape: (0, 0)
┌┐
╞╡
└┘

### drop mv

In [ ]:
query_text = """--sql
    DROP TABLE db1.requests_st_support_mv
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘


### drop ch

In [93]:
query_text = """--sql
    DROP TABLE db1.requests_st_support_ch
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### refresh mv

In [2]:
query_text = """
SYSTEM REFRESH VIEW db1.requests_st_support_mv
"""

ch.query_run(query_text)

elapsed_ns,query_id
i64,str
890357,"""62108f52-061c-4c85-855c-119f54…"
